# Use of Sequential Chain to generate a random review and then moderate it
The purpose of this experiment is to refine the moderation prompt. First, LLM is used to generate a random review. The review may contain certain disqualifying criteria which the subsequent moderation should reject.

In [43]:

from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

llm = ChatOpenAI(temperature=1)


In [55]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

import random

class PermutationGenerator:
    car = None
    unprofessional_statuses = []
    pets = None
    family = None
    choice_numerals = ["2", "3", "4"]
    pet_kinds = ["dogs", "cats", "pet pytons"]
    choice_cars = [
        "2014 Toyota Camry",
        "2019 Ram 1500",
        "2020 Mazda Miata"
    ]
    choice_pets = ["no pets"] + [f"{numeral} {pet}" for numeral in choice_numerals for pet in pet_kinds]
    choice_unprofessional_options = ["Give your address and phone number in case anyone wants to contact you. ", "Use unprofessional tone and some slurs in your review. ", "Mention your personal business by name. ", "Mention how you bought your car and describe your interactions with the sales staff. ", "Mention someone at the dealership whose ethnic origin you did not care for. ", "Mention how your car was serviced by an incompetent mechanic. ", "Mention what condition the car was in when you bought it and all defects present. "]
    choice_family = ["single"] +  [f"a single mom of {numeral}" for numeral in choice_numerals] + [f"a mother of {numeral}" for numeral in choice_numerals] + [f"a single father of {numeral}" for numeral in choice_numerals] + [ f"a father of {numeral}" for numeral in choice_numerals]
    def unprofessional_status_as_string(self) -> str:
        return ''.join(self.unprofessional_statuses)
    def generate_next(self):
        self.car = random.choice(self.choice_cars)
        self.unprofessional_statuses = random.sample(self.choice_unprofessional_options, random.choice(range(0, len(self.choice_unprofessional_options))))
        self.pets = random.choice(self.choice_pets)
        self.family = random.choice(self.choice_family)

class GuidelineViolatingReviewPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        prompt_template = PromptTemplate.from_template("Write a car review of {car}. You are {familialStatus}. You have {petStatus}. {unprofessionalStatuses}Keep it short and informal, using adjectives sparsely.\n\nYour response:")
        permutation_generator = kwargs["permutation_generator"]
        permutation_generator.generate_next()
        result = prompt_template.format(car=permutation_generator.car, familialStatus = permutation_generator.family, petStatus = permutation_generator.pets, unprofessionalStatuses = permutation_generator.unprofessional_status_as_string())
        return result



In [56]:
from langchain.chains import LLMChain
chain = LLMChain(llm=ChatOpenAI(temperature=1), prompt=GuidelineViolatingReviewPromptTemplate(input_variables=["permutation_generator"]))


In [57]:
moderation_prompt = """
You are an online content moderation expert whose purpose is to classify vehicle reviews. Classification is either "Accepted" or "Rejected."
A review can get rejected if it contains inappropriate language, including profanities or slurs. In this case, the rejection reason is: "profanity." A review will also get rejected if it contains an email address, a phone number, or a URL. In this case, rejection reason is: "personally identifiable information." Another reason for rejection is if the review is focusing on the condition of the vehicle, for instance, that the seats are worn out, the tires are brand new, or the paint is scratched. Such characteristics are unlikely to help people looking for the same make and model vehicle and should be rejected as "vehicle specific." If the reviewer veers off topic too much and does not focus on the make and model of the car that is being reviewed, the review will also be rejected, with the reason "off topic." Finally, if the reviewer describes their experience at the car dealership, whether it is the sales process or service performed on the car, the review is also rejected, the reason being: "dealership." In all other cases, the review is accepted. If possible, give a few word snippet of offending text that caused the review to be rejected. Now, classify a vehicle review below delimited by triple backticks.
```{review}```
First, summarize the review. Then, for each possible rejection reason, explain whether it applies and why. Explain your decision, even if the reason does not apply. Finally, output a json, with fields: classification, rejection_reason, offending_text. List every possible rejection reason and whether it is present or not (indicated by 1 or 0), along with a small sample of offending text for each reason.
"""

moderation_chain = LLMChain(llm=ChatOpenAI(temperature=0), prompt = PromptTemplate(input_variables = ["review"], template = moderation_prompt))


In [59]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, moderation_chain], verbose=True)
permutation_generator = PermutationGenerator()
review = overall_chain.run(permutation_generator)
print(f"Review permutation:\n{permutation_generator.car}\n{permutation_generator.unprofessional_statuses}\n")
print(review)



> Entering new SimpleSequentialChain chain...
Title: My "Unimpressive" 2019 Ram 1500 Review - Mama the Python Lady

Hey y'all! Mama the Python Lady here, ready to spill the beans on my 2019 Ram 1500. Now, don't expect no fancy-pants professional review, 'cause I'm just a regular mom with a biz to run and a couple of slithering sidekicks.

First off, let me tell ya, my personal business, Snakeville Reptiles, ain't got nothin' to do with this pitiful excuse for a truck. Got it serviced by the biggest certified dummy of a mechanic, who couldn't fix a toast if his life depended on it. Guess what? My beloved Ram came out more screwed up than my ex's brain!

Bought this beauty, or should I say, beast, off a shady lot. They promised me it was in pristine condition, but boy, were they pullin' a fast one on me! This roarin' wreck had more defects than I got snakes in my collection. From dodgy brakes to leaky ol' pipelines, it was a clustertruck of nightmares!

Now, let's talk about those sale